In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.datasets import load_files
from keras.utils import np_utils
from glob import glob
def load_data(path):
    data = load_files(path)
    X = np.array(data['filenames'])
    Y = np.array(data['target'])
    Y_names = data['target_names']
    return X,Y,Y_names

In [ ]:
X_data , Y_data, food_names = load_data('../input/food-101/food-101/food-101/images')

In [ ]:
print("Food Types :", food_names)
food_names.sort()

In [ ]:
print(X_data[0])

In [ ]:
print("Total number of files : ", X_data.shape)

In [ ]:
import cv2
import matplotlib.pyplot as plt
def plot_images(photos) : #to plot multiple image
    x=0
    for image in photos:
        image_cv = cv2.imread(image)
        plt.subplot(5, 5, x+1)
        plt.imshow(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB));
        plt.axis('off');
        x+=1
plot_images(X_data[:25])

In [ ]:
def view_images(image): #function to view an image
    image_cv = cv2.imread(image) #reads an image
    ax = plt.figure()
    plt.imshow(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB)); #displays an image
view_images(X_data[5])
print(food_names[Y_data[5]])

In [ ]:
from keras.utils  import to_categorical
Y_data_encoded=to_categorical(Y_data)

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data_encoded, test_size = 0.30, random_state = 42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.20, random_state = 42)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
from keras.applications.resnet50 import ResNet50
x_train = paths_to_tensor(X_train)
x_train_preprocessed = preprocess_input(x_train)
x_valid = paths_to_tensor(X_valid)
x_valid_preprocessed = preprocess_input(x_valid)
x_test = paths_to_tensor(X_test)
x_test_preprocessed = preprocess_input(x_test)
model = ResNet50(weights='imagenet',include_top=False)

In [ ]:
resnet50 = Sequential()
resnet50.add(GlobalAveragePooling2D(input_shape=x_train.shape[1:]))
resnet50.add(Dense(1024,activation='relu'))
resnet50.add(Dropout(0.6))
resnet50.add(Dense(512,activation='relu'))
resnet50.add(Dropout(0.45))
resnet50.add(Dense(256,activation='relu'))
resnet50.add(Dropout(0.3))
resnet50.add(Dense(101, activation='softmax'))
Inception_model_wa.summary()

In [ ]:
x_train_resnet = model.predict(x_train_preprocessed)
x_valid_resnet = model.predict(x_valid_preprocessed)
x_test_resnet = model.predict(x_test_preprocessed)

In [ ]:
val_acc = float('-inf')
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(x_train_preprocessed, y_train, batch_size=32):
        resnet50.fit(x_data_resnet, y_batch)
        y_value_true = resnet50.predict(x_valid_resnet)
        predictions = [np.argmax(resnet50.predict(feature)) for feature in x_valid_resnet]
        valid = 100*np.sum(np.array(predictions)==np.argmax(y_valid, axis=1))/len(predictions)
        print('valid accuracy : ',valid*100)
        filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
        callbacks_list = [checkpoint]
        if(valid>=valid_acc):
            val_acc = valid
        batches += 1
        if batches >= len(x_train) / 32:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

In [ ]:
rom PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')
valid_tensors = paths_to_tensor(valid_files).astype('float32')
test_tensors = paths_to_tensor(test_files).astype('float32')